# Define class Player


In [1]:
class Player:
    def __init__(self, name, id_):
        self.summoner_id = id_
        self.summoner_name = name
        self.champion_mastery = {}
        self.champion_count = 0
        self.win_rate_sum = 0
    def add_champion_mastery(self, champion, rate, match):
        self.champion_mastery[champion] = [rate, match]
        if match > 4:
            self.champion_count += 1
            self.win_rate_sum += rate
    def print_champion_mastery(self):
        print("==================="+self.summoner_name+"===================")
        print("Average win rate: "+str(self.avg_win_rate()))
        for key, val in self.champion_mastery.items():
            print(key, val)
    def avg_win_rate(self):
        return self.win_rate_sum/self.champion_count if self.champion_count!=0 else "NA"

# Define class Team

In [2]:
class Team:
    def __init__(self, summoner_list):
        self.summoner_list = summoner_list
        self.champion_list = {}
        for p in summoner_list:
            for c, win_match in p.champion_mastery.items():
                if c in self.champion_list:
                    self.champion_list[c][p.summoner_name] = win_match
                else:
                    self.champion_list[c]={p.summoner_name:win_match}
    def print_pc_chart(self, c_list):
        # print member win rate data
        self.p_c_chart = []
        for p in self.summoner_list:
            self.p_c_chart.append([p.summoner_name])
            for c in c_list:
                if c in p.champion_mastery:
                    self.p_c_chart[-1].append(p.champion_mastery[c])
                else:
                    self.p_c_chart[-1].append(["NA" , 0])
        
        from tabulate import tabulate
        data = []
        for r in self.p_c_chart:
            row = [r[0]]
            for w_m in r[1:]:
                win, match = w_m[0], w_m[1]
                s = ""
                if win == "NA":
                    s += win
                else:
                    if win > 70:
                        s += "* "
                    elif win < 22:
                        s += "! "
                
                    s += str(win)+"%"
                    if match <= 4:
                        s += " ("+str(match)+")" 
                row.append(s)
            data.append(row)
                
        print (tabulate(data, headers=[" "]+c_list))
        
    def current_win_rate(self, c_choice):
        assert len(c_choice) == len(self.summoner_list)
        
        win_sum = 0
        for i, c in enumerate(c_choice):
            avg = self.summoner_list[i].avg_win_rate()
            avg = round(avg,2) if avg!="NA" else 0
            
            if c in self.summoner_list[i].champion_mastery:
                win, match = self.summoner_list[i].champion_mastery[c][0], self.summoner_list[i].champion_mastery[c][1] 
                if match < 5:
                    win_sum += int(float(win)*float(match)/5. + float(avg)*float(5-match)/5.)
                else:
                    win_sum += win
            else:
                win_sum += avg
                
        return win_sum / len(c_choice)
    
    def print_combo_details(self, c_list):
        assert len(c_list) == len(self.summoner_list)
        from tabulate import tabulate
        
        data = []
        for i, p in enumerate(self.summoner_list):
            s, c = "" , c_list[i] 
            row = [p.summoner_name, c]
            avg = p.avg_win_rate()
            avg = round(avg,2) if avg!="NA" else 0
            if c in p.champion_mastery:
                win, match = p.champion_mastery[c][0], p.champion_mastery[c][1]
                s += str(win)+"% Win for "+str(match)+" games "
                if match < 5:
                    s += "& "+str(avg)+"% Average Win rate"
                
            else:
                s += str(avg)+"% Average Win rate"
            
            row.append(s)
            data.append(row)
        print (tabulate(data, headers=["Summoner Name", "Champion", "Details"]))
        print("Win Rate: "+str(round(self.current_win_rate(c_list),2)))
        
    def best_combo(self, c_list):
        assert len(c_list) >= len(self.summoner_list)
        
        # generate all possible combo
        def bc(so_far, remain, count, result):
            if len(so_far) == count:
                result.append(so_far)
                return 
        
            for i in range(len(remain)):
                bc(so_far+[remain[i]], remain[:i]+remain[i+1:], count, result)
               
        combo = []
        bc([], c_list, len(self.summoner_list), combo)
        
        import heapq
        best = []
        for com in combo[:3]:
            rate = self.current_win_rate(com)
            heapq.heappush(best, (rate, com))
            
        for com in combo[3:]:
            rate = self.current_win_rate(com)
            heapq.heappushpop(best, (rate, com))
            
        sort_best = []
        while best:
            sort_best.append(heapq.heappop(best))
        return sort_best[::-1]
    
    def print_best_combo_details(self, c_list):
        best_combos = self.best_combo(c_list) 
        for i, b in enumerate(best_combos):
            print("================================Combo "+str(i+1)+"===============================")
            self.print_combo_details(b[1])

# Parse from file html and scrape mastery score

In [3]:
def parse_html(file_name):
# input: html file location/name.html
# output: parsed Player object
    print("Processing file: "+file_name)

    with open(file_name) as f:
        lines = f.readlines()
        # only consider normal game plays
        # other type: season 2021, season 2019, etc
        normal = []
        start = False
        for l in lines:
            if "<span class=\"Name\">" in l:
                name = l.split("<span class=\"Name\">")[1].split("<")[0]
            if "season=normal" in l or "tabItem normal" in l:
                start = True
            if start and "</div>" == l:
                start = False
            if start:
                normal.append(l)
            if "summonerId=" in l:
                id_ = l.split("summonerId=")[1].split('&')[0]
        print("Found name: "+name+" Id: "+ id_)
        
        
        # process normal game champion win rate
        c_start = False
        summoner = Player(name, id_)
        for l in normal:
            if '\t\t\t\t<td class="ChampionName Cell" data-value=' in l:
                c_start = True
                champion = l.split('"')[3].replace('&#39;','\'')
                win, lose, win_rate = 0, 0, 0
            if c_start and '\t\t\t\t\t\t\t<span class="WinRatio' in l:
                c_start = False
                win_rate = int(l.split('>')[1].split('%')[0])
                summoner.add_champion_mastery(champion, win_rate, win+lose)
            if c_start and '\t\t\t\t\t\t\t\t\t<div class="Text Left">' in l:
                win = int(l.split('>')[1].split('W')[0])

            if c_start and '\t\t\t\t\t\t\t\t\t<div class="Text Right">' in l:
                lose = int(l.split('>')[1].split('L')[0])
        print("Average win rate: "+str(summoner.avg_win_rate()))
    return summoner


# Form Team and get player champion relations

In [4]:
import os
# Scan file from directory and open them
summoner_list = []
entries = os.listdir('./')
for entry in entries:
    if ".html" in entry:
        summoner_list.append(parse_html(entry))

Processing file: MageTerra.html
Found name: MageTerra Id: 34663269
Average win rate: 48.22222222222222
Processing file: TheHappyLeague.html
Found name: TheHappyLeague Id: 37035766
Average win rate: 50.52
Processing file: height190.html
Found name: height190 Id: 108361524
Average win rate: 54.57142857142857
Processing file: IWasAlready.html
Found name: IWasAlready Id: 65049997
Average win rate: 42.0
Processing file: meowtella.html
Found name: meowtella Id: 107194196
Average win rate: 46.35294117647059


# Create Team
`t = Team(summoner_list)`

# Dice Roll results 

`dice_roll_list = ['Lillia','Veigar','Vel\'Koz','Caitlyn','Fizz','Vayne','Jinx', 'Twisted Fate', 'Amumu']
t.print_pc_chart(dice_roll_list)`

# Best 3 combo 

`dice_roll_list2 = ['Lillia','Veigar','Vel\'Koz','Caitlyn','Neeko','Vayne','Jinx', 'Twisted Fate', 'Amumu','Warwick']
t.print_pc_chart(dice_roll_list2)
t.print_best_combo_details(dice_roll_list2)`

# View Complete Champion mastery list of current team

`t.champion_list`

# Complete Champion List

In [5]:
champion_list = ["Aatrox","Ahri","Akali","Akshan","Alistar","Amumu","Anivia","Annie","Aphelios","Ashe","Aurelion Sol","Azir","Bard","Blitzcrank","Brand","Braum","Caitlyn","Camille","Cassiopeia","Cho\'Gath","Corki","Darius","Diana","Dr. Mundo","Draven","Ekko","Elise","Evelynn","Ezreal","Fiddlesticks","Fiora","Fizz","Galio","Gangplank","Garen","Gnar","Gragas","Graves","Gwen","Hecarim","Heimerdinger","Illaoi","Irelia","Ivern","Janna","Jarvan IV","Jax","Jayce","Jhin","Jinx","Kai\'Sa","Kalista","Karma","Karthus","Kassadin","Katarina","Kayle","Kayn","Kennen","Kha\'Zix","Kindred","Kled","Kog\'Maw","Leblanc","Lee Sin","Leona","Lillia","Lissandra","Lucian","Lulu","Lux","Malphite","Malzahar","Maokai","Master Yi","Miss Fortune","Mordekaiser","Morgana","Nami","Nasus","Nautilus","Neeko","Nidalee","Nocturne","Nunu & Willump","Olaf","Orianna","Ornn","Pantheon","Poppy","Pyke","Qiyana","Quinn","Rakan","Rammus","Rek\'Sai","Rell","Renekton","Rengar","Riven","Rumble","Ryze","Samira","Sejuani","Senna","Seraphine","Sett","Shaco","Shen","Shyvana","Singed","Sion","Sivir","Skarner","Sona","Soraka","Swain","Sylas","Syndra","Tahm Kench","Taliyah","Talon","Taric","Teemo","Thresh","Tristana","Trundle","Tryndamere","Twisted Fate","Twitch","Udyr","Urgot","Varus","Vayne","Veigar","Vel\'Koz","Vex","Vi","Viego","Viktor","Vladimir","Volibear","Warwick","Wukong","Xayah","Xerath","Xin Zhao","Yasuo","Yone","Yorick","Yuumi","Zac","Zed","Ziggs","Zilean","Zoe","Zyra"]

# Confirm Player

In [6]:
s = ""
for i, p in enumerate(summoner_list):
    s += str(i+1)+"."+p.summoner_name+" "
print("Current Players: "+s)
val = input("Remove any? (Press 'Enter' for no)")
if val != "":
    summoner_list.pop(int(val)-1)
    
t = Team(summoner_list)

Current Players: 1.MageTerra 2.TheHappyLeague 3.height190 4.IWasAlready 5.meowtella 
Remove any? (Press 'Enter' for no)


# New Game: Get Dice roll results

In [7]:
end = False
dice_roll = []

while not end and len(dice_roll) < 15:
    val = input("Champion "+str(len(dice_roll)+1)+": ")
    if val == "":
        end = True
        print("==============")
        print("Current dice rolls:")
        d = ""
        for i, s in enumerate(dice_roll):
            d += str(i+1)+"."+s+" "
        num = input(d+"\n\tPress 'Enter' to continue. \n To Change any champions, reply number")  
        if num == "":
            break
        else:
            index = int(num)-1
            dice_roll.pop(index)
            end = False
    else:
        suggest = []
        for c in champion_list:
            if val.casefold() in c.casefold() :
                suggest.append(c)
                
        if len(suggest)==1:
            print("\tAdded "+suggest[0])
            dice_roll.append(suggest[0])
        else:
            s1 = ""
            for i, s in enumerate(suggest):
                s1 += str(i+1)+"."+s+" "
            if s1 == "":
                print("No match")
                count -= 1
            else:
                opt = input("\tDo you mean:" + s1+"? Reply number: ")
                index = int(opt)-1
                dice_roll.append(suggest[index])
                
print("####################################")
print("# Best Combos")
print("####################################")

t.print_best_combo_details(dice_roll)
print("\n\n")
print("####################################")
print("# Player Champion Chart")
print("####################################")

t.print_pc_chart(dice_roll)

Champion 1: mort
No match
Champion 1: mo
	Do you mean:1.Mordekaiser 2.Morgana 3.Teemo ? Reply number: 2
Champion 2: fixzz
No match
Champion 2: fizz
	Added Fizz
Champion 3: cai
	Added Caitlyn
Champion 4: vel
	Do you mean:1.Evelynn 2.Vel'Koz ? Reply number: 2
Champion 5: cho
	Added Cho'Gath
Champion 6: mal
	Do you mean:1.Malphite 2.Malzahar ? Reply number: 1
Champion 7: amu
	Added Amumu
Champion 8: vay
	Added Vayne
Champion 9: illa
	Added Illaoi
Champion 10: lili
No match
Champion 10: li
	Do you mean:1.Akali 2.Alistar 3.Aphelios 4.Aurelion Sol 5.Blitzcrank 6.Elise 7.Galio 8.Irelia 9.Kalista 10.Lillia 11.Lissandra 12.Taliyah 13.Volibear ? Reply number: 10
Champion 11: 
Current dice rolls:
1.Morgana 2.Fizz 3.Caitlyn 4.Vel'Koz 5.Cho'Gath 6.Malphite 7.Amumu 8.Vayne 9.Illaoi 10.Lillia 
	Press 'Enter' to continue. 
 To Change any champions, reply number
####################################
# Best Combos
####################################
================================Combo 1===============

# Reroll a dice

In [10]:
end = False
s = ""
for i, c in enumerate(dice_roll):
    s += str(i+1)+"."+c+" "
print("Current Dice rolls:" + s)
while not end and len(dice_roll) < 15:
    val = input("Champion "+str(len(dice_roll)+1)+": ")
    if val == "":
        end = True
        print("==============")
        print("Current dice rolls:")
        d = ""
        for i, s in enumerate(dice_roll):
            d += str(i+1)+"."+s+" "
        num = input(d+"\n\tPress 'Enter' to continue. \n To Change any champions, reply number")  
        if num == "":
            break
        else:
            index = int(num)-1
            dice_roll.pop(index)
            end = False
    else:
        suggest = []
        for c in champion_list:
            if val.casefold() in c.casefold() :
                suggest.append(c)
                
        if len(suggest)==1:
            print("\tAdded "+suggest[0])
            dice_roll.append(suggest[0])
        else:
            s1 = ""
            for i, s in enumerate(suggest):
                s1 += str(i+1)+"."+s+" "
            if s1 == "":
                print("No match")
                count -= 1
            else:
                opt = input("\tDo you mean:" + s1+"? Reply number: ")
                index = int(opt)-1
                dice_roll.append(suggest[index])
                
print("####################################")
print("# Best Combos")
print("####################################")

t.print_best_combo_details(dice_roll)
print("\n\n")
print("####################################")
print("# Player Champion Chart")
print("####################################")

t.print_pc_chart(dice_roll)

Current Dice rolls:1.Morgana 2.Fizz 3.Caitlyn 4.Vel'Koz 5.Cho'Gath 6.Malphite 7.Amumu 8.Vayne 9.Illaoi 10.Lillia 11.Elise 
Champion 12: 
Current dice rolls:
1.Morgana 2.Fizz 3.Caitlyn 4.Vel'Koz 5.Cho'Gath 6.Malphite 7.Amumu 8.Vayne 9.Illaoi 10.Lillia 11.Elise 
	Press 'Enter' to continue. 
 To Change any champions, reply number5
Champion 11: 
Current dice rolls:
1.Morgana 2.Fizz 3.Caitlyn 4.Vel'Koz 5.Malphite 6.Amumu 7.Vayne 8.Illaoi 9.Lillia 10.Elise 
	Press 'Enter' to continue. 
 To Change any champions, reply number
####################################
# Best Combos
####################################
================================Combo 1===============================
Summoner Name    Champion    Details
---------------  ----------  ----------------------------------------------
MageTerra        Vayne       100% Win for 3 games & 48.22% Average Win rate
TheHappyLeague   Illaoi      100% Win for 2 games & 50.52% Average Win rate
height190        Vel'Koz     100% Win for 6 games
IW